In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
import datetime, time, os
from tensorflow.keras.layers.experimental import preprocessing
    
print('Using TensorFlow version: %s' % tf.__version__)
np.set_printoptions(precision=3, suppress=True)

import datetime, time, os
from datetime import datetime

from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, KFold

from src.preprocessing.preprocessing_functions import drop_rows

#!pip install -q git+https://github.com/tensorflow/docs # install first time

RSEED = 42

In [ ]:
# With this command you can clear any logs from previous runs
# If you want to compare different runs you can skip this cell 
!rm -rf my_logs/

## Loading the processed dataframe (sugarbeet and openweather station data)


In [ ]:
df = pd.read_pickle('pickles/df_fieldweather_devstage_sugarbeet.pkl')
df

In [ ]:
# convert to string for categorizing
df['seednames_coded'] = df['seednames_coded'].astype(str)
df['pollinator_comp'] = df['pollinator_comp'].astype(str)
df['ms_comp'] = df['ms_comp'].astype(str)
df['otype_comp'] = df['otype_comp'].astype(str)

In [ ]:
df.shape

In [ ]:
df.station_location.unique()

In [ ]:
# drop locations for comparison
drop_rows(df, 'station_location', ['Bautzen', 'Goderville', 'Lamotte', 'Lelystad', 'Pithiviers', 
                                   'Rittershausen', 'Sommepy', 'Vierhoefen'])

In [ ]:
# dropping columns 
df.drop(['betaine_nir', 
         'cry_nir', 
         'dm_nir', 
         'invert_nir', 
         'mark_nir', 
         #'sc_nir',
         'csy_nir', 
         'totaln_nir',
         'obj',  
         'seriesid', 
         'x', 
         'y', 
         'ms_comp',
         'otype_comp', 
         #'pollinator_comp',
         #'seednames_coded',
         'region',
         'station_location'
         ], axis=1, inplace=True)
df.columns


In [ ]:
df.shape

In [ ]:
# reset index after drpping columns
df = df.reset_index(drop=True)

#### Creating the pipeline

In [ ]:
# creating list for categorical predictors/features 
cat_features = list(df.columns[df.dtypes==object])
cat_features

In [ ]:
# creating list for numerical predictors/features
# since 'sc_nir' is our target variable we will exclude this feature from the list of numerical predictors 
# latitude and longitude are also excluded to avoid location influence on prediction
num_features = [
 'air_temperature_avg_devstage_1',
 'air_temperature_avg_devstage_2',
 'air_temperature_avg_devstage_3',
 'air_temperature_max_devstage_1',
 'air_temperature_max_devstage_2',
 'air_temperature_max_devstage_3',
 'air_temperature_min_devstage_1',
 'air_temperature_min_devstage_2',
 'air_temperature_min_devstage_3',
 'deltat_avg_devstage_1',
 'deltat_avg_devstage_2',
 'deltat_avg_devstage_3',
 'deltat_max_devstage_1',
 'deltat_max_devstage_2',
 'deltat_max_devstage_3',
 'deltat_min_devstage_1',
 'deltat_min_devstage_2',
 'deltat_min_devstage_3',
 'dew_point_avg_devstage_1',
 'dew_point_avg_devstage_2',
 'dew_point_avg_devstage_3',
 'dew_point_min_devstage_1',
 'dew_point_min_devstage_2',
 'dew_point_min_devstage_3',
 'eag_soil_moisture_1_devstage_1',
 'eag_soil_moisture_1_devstage_2',
 'eag_soil_moisture_1_devstage_3',
 'eag_soil_moisture_2_devstage_1',
 'eag_soil_moisture_2_devstage_2',
 'eag_soil_moisture_2_devstage_3',
 'eag_soil_moisture_3_devstage_1',
 'eag_soil_moisture_3_devstage_2',
 'eag_soil_moisture_3_devstage_3',
 'eag_soil_moisture_4_devstage_1',
 'eag_soil_moisture_4_devstage_2',
 'eag_soil_moisture_4_devstage_3',
 'eag_soil_moisture_5_devstage_1',
 'eag_soil_moisture_5_devstage_2',
 'eag_soil_moisture_5_devstage_3',
 'eag_soil_moisture_6_devstage_1',
 'eag_soil_moisture_6_devstage_2',
 'eag_soil_moisture_6_devstage_3',
 'et0_devstage_1',
 'et0_devstage_2',
 'et0_devstage_3',
#  'latitude_1',
#  'latitude_2',
#  'latitude_3',
 'leaf_wetness_devstage_1',
 'leaf_wetness_devstage_2',
 'leaf_wetness_devstage_3',
#  'longitude_1',
#  'longitude_2',
#  'longitude_3',
 'precipitation_devstage_1',
 'precipitation_devstage_2',
 'precipitation_devstage_3',
 'relative_humidity_avg_devstage_1',
 'relative_humidity_avg_devstage_2',
 'relative_humidity_avg_devstage_3',
 'relative_humidity_max_devstage_1',
 'relative_humidity_max_devstage_2',
 'relative_humidity_max_devstage_3',
 'relative_humidity_min_devstage_1',
 'relative_humidity_min_devstage_2',
 'relative_humidity_min_devstage_3',
 'saturation_vpd_avg_devstage_1',
 'saturation_vpd_avg_devstage_2',
 'saturation_vpd_avg_devstage_3',
 'saturation_vpd_min_devstage_1',
 'saturation_vpd_min_devstage_2',
 'saturation_vpd_min_devstage_3',
 'soil_salinity_1_devstage_1',
 'soil_salinity_1_devstage_2',
 'soil_salinity_1_devstage_3',
 'soil_salinity_2_devstage_1',
 'soil_salinity_2_devstage_2',
 'soil_salinity_2_devstage_3',
 'soil_salinity_3_devstage_1',
 'soil_salinity_3_devstage_2',
 'soil_salinity_3_devstage_3',
 'soil_salinity_4_devstage_1',
 'soil_salinity_4_devstage_2',
 'soil_salinity_4_devstage_3',
 'soil_salinity_5_devstage_1',
 'soil_salinity_5_devstage_2',
 'soil_salinity_5_devstage_3',
 'soil_salinity_6_devstage_1',
 'soil_salinity_6_devstage_2',
 'soil_salinity_6_devstage_3',
 'soil_temperature_1_max_devstage_1',
 'soil_temperature_1_max_devstage_2',
 'soil_temperature_1_max_devstage_3',
 'soil_temperature_1_min_devstage_1',
 'soil_temperature_1_min_devstage_2',
 'soil_temperature_1_min_devstage_3',
 'soil_temperature_1_vg_devstage_1',
 'soil_temperature_1_vg_devstage_2',
 'soil_temperature_1_vg_devstage_3',
 'soil_temperature_2_max_devstage_1',
 'soil_temperature_2_max_devstage_2',
 'soil_temperature_2_max_devstage_3',
 'soil_temperature_2_min_devstage_1',
 'soil_temperature_2_min_devstage_2',
 'soil_temperature_2_min_devstage_3',
 'soil_temperature_2_vg_devstage_1',
 'soil_temperature_2_vg_devstage_2',
 'soil_temperature_2_vg_devstage_3',
 'soil_temperature_3_max_devstage_1',
 'soil_temperature_3_max_devstage_2',
 'soil_temperature_3_max_devstage_3',
 'soil_temperature_3_min_devstage_1',
 'soil_temperature_3_min_devstage_2',
 'soil_temperature_3_min_devstage_3',
 'soil_temperature_3_vg_devstage_1',
 'soil_temperature_3_vg_devstage_2',
 'soil_temperature_3_vg_devstage_3',
 'soil_temperature_4_max_devstage_1',
 'soil_temperature_4_max_devstage_2',
 'soil_temperature_4_max_devstage_3',
 'soil_temperature_4_min_devstage_1',
 'soil_temperature_4_min_devstage_2',
 'soil_temperature_4_min_devstage_3',
 'soil_temperature_4_vg_devstage_1',
 'soil_temperature_4_vg_devstage_2',
 'soil_temperature_4_vg_devstage_3',
 'soil_temperature_5_max_devstage_1',
 'soil_temperature_5_max_devstage_2',
 'soil_temperature_5_max_devstage_3',
 'soil_temperature_5_min_devstage_1',
 'soil_temperature_5_min_devstage_2',
 'soil_temperature_5_min_devstage_3',
 'soil_temperature_5_vg_devstage_1',
 'soil_temperature_5_vg_devstage_2',
 'soil_temperature_5_vg_devstage_3',
 'soil_temperature_6_max_devstage_1',
 'soil_temperature_6_max_devstage_2',
 'soil_temperature_6_max_devstage_3',
 'soil_temperature_6_min_devstage_1',
 'soil_temperature_6_min_devstage_2',
 'soil_temperature_6_min_devstage_3',
 'soil_temperature_6_vg_devstage_1',
 'soil_temperature_6_vg_devstage_2',
 'soil_temperature_6_vg_devstage_3',
 'solar_radiation_devstage_1',
 'solar_radiation_devstage_2',
 'solar_radiation_devstage_3',
 'wind_direction_devstage_1',
 'wind_direction_devstage_2',
 'wind_direction_devstage_3',
 'wind_speed_avg_devstage_1',
 'wind_speed_avg_devstage_2',
 'wind_speed_avg_devstage_3',
 'wind_speed_gusts_devstage_1',
 'wind_speed_gusts_devstage_2',
 'wind_speed_gusts_devstage_3',
 'wind_speed_max_devstage_1',
 'wind_speed_max_devstage_2',
 'wind_speed_max_devstage_3'
]
num_features

In [ ]:
# define predictors and target variable
X = df.drop('sc_nir', axis=1)
y = df['sc_nir']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

In [ ]:
# split into train and test set (train set: 70%, test set: 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RSEED) 

#### Preprocessing Pipeline

In [ ]:
# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# pipeline for numerical features
num_pipeline = Pipeline([
    ('scaler', RobustScaler())
])

In [ ]:
# complete pipeline for numerical features
# apply transformers to numerical pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
], sparse_threshold=0)

#### Transform X_train

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
X_train_transformed.shape

X_tf_train = tf.convert_to_tensor(X_train_transformed)
y_tf_train = tf.convert_to_tensor(y_train)

X_tf_test = tf.convert_to_tensor(X_test_transformed)
y_tf_test = tf.convert_to_tensor(y_test)


## Training
For training you need a train/val split (hopefully you did a train/test split before (and you should use the same as in your ML project to make results comparable). 

In [ ]:
# define dictionary to store results
training_history = {}

# define number of epochs and learning rate decay
N_TRAIN = len(X_train)
N_VAL = 0.2
EPOCHS = 100
BATCH_SIZE = N_TRAIN // 10
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
#     0.001,
#     decay_steps=STEPS_PER_EPOCH*1000,
#     decay_rate=1,
#     staircase=False)

### Build, compile and fit your model


In [ ]:
# define path for new directory 
root_logdir = os.path.join(os.curdir, "my_logs")

# define function for creating a new folder for each run
def get_run_logdir():
    now = datetime.now()
    run_id = now.strftime('%Y-%m-%d %H:%M:%S')
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [ ]:
# define path where checkpoints should be stored
checkpoint_path = "training_1/ML_model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0) # Set verbose != 0 if you want output during training 
# return [list of your callbacks]
def get_callbacks(name):
    return tf.keras.callbacks.TensorBoard(run_logdir+name, histogram_freq=1)

You can implement your callbacks in the `model.fit()` method below.

In [ ]:
def model_compile_and_fit(model, name, optimizer=None, max_epochs=EPOCHS):
  
    # model.compile
    model.compile(optimizer = 'adam', loss = 'mae', metrics = ['mse'])
    
    # model.fit
    history = model.fit(X_tf_train, y_train, batch_size = BATCH_SIZE, validation_split=N_VAL, epochs = max_epochs, callbacks=get_callbacks(name))
    
    # return results
    return history

In [ ]:
X_train.shape

#### Build your model
You can build your model by using `tf.keras.Sequential()` that helps you to sequentially define your different layers from input to output. 

## First model

In [ ]:
with tf.device('/cpu:0'):
    model = tf.keras.Sequential([
        layers.Dense(100, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(1)
        ])


#### Train your model
Train your model by using your `model_compile_and_fit()` function you defined above.

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=RSEED)
cvscores = []

for train, test in kfold.split(X_tf_train,y_tf_train):

#your_history = model_compile_and_fit(your_model, ....)
    with tf.device('/cpu:0'):
        training_history["first_model"] = model_compile_and_fit(model, "first_model")

model.summary()

#### Evaluate your model training
TensorFlow offers now (this was more cumbersome before) a simple history plotter that you can use to plot training histories and see how the model performed on training and validation data set.

In [ ]:
training_history

In [ ]:
# plotting function for MSE
def plot_metric(history):
    plt.plot(history.history['mse'])
    plt.plot(history.history['val_mse'])
    plt.ylim([0, 2.5])
    plt.title('Model MSE')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()

In [ ]:
plot_metric(training_history['first_model'])

In [ ]:
# plotting function for loss
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 5])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(training_history['first_model'])

In [ ]:
history_plotter = tfdocs.plots.HistoryPlotter(metric = 'mse', smoothing_std=10)
history_plotter.plot(training_history)

In [ ]:
training_history

In [ ]:
loss, mse = model.evaluate(X_tf_test, y_tf_test, verbose=0)
print(f'Model MSE: {mse}')

In [ ]:
y_pred = model.predict(X_tf_test).flatten()

In [ ]:
def error_analysis(y_test, y_pred):
    """Generated true vs. predicted values and residual scatter plot for models
    Args:
        y_test (array): true values for y_test
        y_pred_test (array): predicted values of model for y_test
    """
    # calculate residuals
    residuals = y_test - y_pred
    # plot real vs. predicted values
    fig, ax = plt.subplots(1,2, figsize=(15, 5))
    plt.subplots_adjust(right=1)
    plt.suptitle('')
    ax[0].scatter(y_pred, y_test, color="#2f7055", alpha=0.7)
    ax[0].plot([5, 35], [5, 35], color="#193251")
    ax[0].set_title("True vs. predicted values", fontsize=16)
    ax[0].set_xlabel("predicted values", fontsize=14)
    ax[0].set_ylabel("true values",  fontsize=14)
    ax[0].set_xlim((5), (y_pred.max()+10))
    #ax[0].set_ylim((5), (y_test.max()+10))
    ax[0].tick_params(axis='both', which='major', labelsize=16)
    ax[1].scatter(y_pred, residuals, color="#2f7055", alpha=0.7)
    ax[1].plot([-400, 350], [0,0], color="#193251")
    ax[1].set_title("Residual Scatter Plot", fontsize=16)
    ax[1].set_xlabel("predicted values", fontsize=14)
    ax[1].set_ylabel("residuals", fontsize=14)
    ax[1].set_xlim((y_pred.min()-5), (y_pred.max()+5))
    #ax[1].set_ylim((residuals.min()-5), (residuals.max()+5))
    ax[1].tick_params(axis='both', which='major', labelsize=16);

In [ ]:
error_analysis(y_tf_test,y_pred)

## Model tuning

## Second model

In [ ]:
#===========#
# Second Model with more nodes
#===========#

with tf.device('/cpu:0'):
    model = tf.keras.Sequential([
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(1)
    ])  


kfold = KFold(n_splits=5, shuffle=True, random_state=RSEED)
cvscores = []

for train, test in kfold.split(X_tf_train,y_tf_train):
#your_history = model_compile_and_fit(your_model, ....)
    with tf.device('/cpu:0'):
        training_history["second_model"] = model_compile_and_fit(model, "second_model")

model.summary()

In [ ]:
plot_metric(training_history['second_model'])

In [ ]:
plot_loss(training_history['second_model'])

In [ ]:
history_plotter = tfdocs.plots.HistoryPlotter(metric = 'mse', smoothing_std=10)
history_plotter.plot(training_history)

In [ ]:
training_history

In [ ]:
loss, mse = model.evaluate(X_tf_test, y_tf_test, verbose=2)
print(f'Model MSE: {mse}')

In [ ]:
y_pred = model.predict(X_tf_test).flatten()

In [ ]:
error_analysis(y_tf_test,y_pred)

## Third model

In [ ]:
#===========#
# Third Model with more layers
#===========#

with tf.device('/cpu:0'):
    model = tf.keras.Sequential([
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        # layers.Dense(500, activation='relu'),
        layers.Dense(1)
    ])  

kfold = KFold(n_splits=5, shuffle=True, random_state=RSEED)
cvscores = []

for train, test in kfold.split(X_tf_train,y_tf_train):

#your_history = model_compile_and_fit(your_model, ....)
    with tf.device('/cpu:0'):
        training_history["third_model"] = model_compile_and_fit(model, "third_model")

model.summary()        

In [ ]:
plot_metric(training_history['third_model'])

In [ ]:
plot_loss(training_history['third_model'])

In [ ]:
history_plotter = tfdocs.plots.HistoryPlotter(metric = 'mse', smoothing_std=10)
history_plotter.plot(training_history)

In [ ]:
training_history

In [ ]:
loss, mse = model.evaluate(X_tf_test, y_tf_test, verbose=2)
print(f'Model MSE: {mse}')

In [ ]:
y_pred = model.predict(X_tf_test).flatten()

In [ ]:
error_analysis(y_tf_test,y_pred)

## Fourth model

In [ ]:
#===========#
# Fourth Model with dropout
#===========#

with tf.device('/cpu:0'):
    model = tf.keras.Sequential([
        layers.Dense(500, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        layers.Dense(500, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        layers.Dense(1)
    ])  

kfold = KFold(n_splits=5, shuffle=True, random_state=RSEED)
cvscores = []

for train, test in kfold.split(X_tf_train,y_tf_train):

#your_history = model_compile_and_fit(your_model, ....)
    with tf.device('/cpu:0'):
        training_history["fourth_model"] = model_compile_and_fit(model, "fourth_model")

model.summary()        

In [ ]:
plot_metric(training_history['fourth_model'])

In [ ]:
plot_loss(training_history['fourth_model'])

In [ ]:
history_plotter = tfdocs.plots.HistoryPlotter(metric = 'mse', smoothing_std=10)
history_plotter.plot(training_history)

In [ ]:
training_history

In [ ]:
loss, mse = model.evaluate(X_tf_test, y_tf_test, verbose=2)
print(f'Model MSE: {mse}')

In [ ]:
y_pred = model.predict(X_tf_test).flatten()

In [ ]:
error_analysis(y_tf_test,y_pred)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./my_logs --port=6005